# Why codde? 

- ***SigLIP Hyper Tuning***

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
from tqdm import tqdm 
print("Oke")

# Down dataset from the my hub

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset
source_data = load_dataset("QuanHoangNgoc/MS-Flick")
source_data = source_data["test"]
source_data

# Load model: CLIP, SigLIP, Prepare stage: Build Mdb

In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModel, AutoTokenizer
import torch


class SigLipEncoder:
    def __init__(self):
        self.Model = AutoModel.from_pretrained("google/siglip-base-patch16-224")
        self.Processor = AutoProcessor.from_pretrained("google/siglip-base-patch16-224")
        self.Tokenizer = AutoTokenizer.from_pretrained("google/siglip-base-patch16-224")

    def form_model(self, dtype=None, device=None, grad_mode=False, eval_model=True):
        """Moves and converts the model to the specified dtype and device."""
        self.Model.to(dtype=dtype, device=device)
        for param in self.Model.parameters():
            param.requires_grad = grad_mode   
        if(eval_model): 
            self.Model.eval() 
        return self

    def infor_model(self): 
        cnt = 0 
        params = [(name, param) for name, param in self.Model.named_parameters()]
        for name, param in params:
            cnt += 1 
            if(cnt < 3 or cnt > len(params) - 3): 
                print(f"Parameter: {name}")
                print(f"  Data type: {param.dtype}")
                print(f"  Requires gradient: {param.requires_grad}")
                print(f"  Device: {param.device}")
                print("-" * 20)  # Separator for readability

    def form_ts(self, inputs_dict):
        dtype = self.Model.dtype
        device = self.Model.device
        for key, value in inputs_dict.items():
            if isinstance(value, torch.Tensor):
                inputs_dict[key] = value.to(device=device)
                if dtype is not None and value.dtype.is_floating_point: #only convert float tensor
                    inputs_dict[key] = inputs_dict[key].type(dtype)
        return inputs_dict

    def get_np_text(self, str_chunk):
        inputs_ts = self.Tokenizer(str_chunk, padding="max_length", return_tensors="pt")
        inputs_ts = self.form_ts(inputs_ts)
        with torch.inference_mode():
            text_features = self.Model.get_text_features(**inputs_ts).cpu().numpy()  # Move to CPU before converting to NumPy
        return text_features

    def get_np_image(self, pil_chunk):
        inputs_ts = self.Processor(images=pil_chunk, return_tensors="pt")
        inputs_ts = self.form_ts(inputs_ts)
        with torch.inference_mode():
            image_features = self.Model.get_image_features(**inputs_ts).cpu().numpy()  # Move to CPU before converting to NumPy
        return image_features


siglip = SigLipEncoder() 
siglip = siglip.form_model(dtype=torch.float32, device='cuda') 
siglip.infor_model() 

In [ ]:
siglip.get_np_text("people").shape # testcase 

In [ ]:
source_data[0]["image"] # tescase 

In [ ]:
import numpy as np
from tqdm import tqdm

def build_M_db(ds, encoder, batch_size=64):
    """
    Build the M database by encoding images in batches for faster processing.
    
    Args:
        ds: Dataset containing image data.
        encoder: Encoder object with a `get_np_image` method.
        batch_size: Number of images to process per batch.
    
    Returns:
        M: Numpy array of encoded image representations.
    """
    M = []

    # Preprocess data in batches
    for start_idx in tqdm(range(0, len(ds), batch_size), desc="Processing Batches"):
        # Get a batch of images
        batch = [ds[i]["image"] for i in range(start_idx, min(start_idx + batch_size, len(ds)))]

        # Encode the batch (assumes `get_np_image` can process a list of images)
        enc_np_batch = encoder.get_np_image(batch)  # Ensure encoder supports batch processing

        # Add encoded batch to M
        M.append(enc_np_batch)

    # Convert list of batches to a single NumPy array
    M = np.vstack(M)
    return M


Mdb = build_M_db(source_data, siglip) #[warn] long run
print(Mdb.shape)

# Retrieval stage, Iterative feedback loop

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_ordered(query_enc, Mdb):
    """
    Computes cosine similarity between the encoded text and Mdb,
    and returns an ordered list of indices based on similarity.

    Args:
        text (str): The input text.
        encoder (SigLipEncoder): The encoder object.
        Mdb (np.ndarray): The database of encoded vectors.

    Returns:
        np.ndarray: An array of indices sorted by cosine similarity.
    """
    similarities = cosine_similarity(query_enc.reshape(1, -1), Mdb)  # Reshape enc_np for cosine_similarity
    ordered_indices = np.argsort(similarities[0])[::-1]  # Sort in descending order
    return ordered_indices

def representation(text, encoder): 
    query_enc = encoder.get_np_text(text)[0] 
    return query_enc 

# q = representation("people", siglip)
# order = get_ordered(q, Mdb)

In [ ]:
def get_centroid(x):
    if x is None:
        return None
    if isinstance(x, list):
        return np.stack(x, dim = 0).mean(0)
    else:
    # elif isinstance(x, np.) and x.ndim == 2:
        return np.mean(x, 0)
    
def rocchio_relevance_feedback(query = None, positive = None, negative = None, alpha = 1, beta = 0.8, gamma = 0.1):
    """
        Rocchio algorithm for relevance feedback as follows:
            newQuery = alpha * query + beta * centroid(positive) - gamma * centroid(negative)
        Args:
            query:
            positive:
            negative:
            alpha, beta, gamma: 
        Returns: 
            newQuery: 
    """
    # print("Rocchio relevance feedback", end=" [>] ")
    
    newQuery = query * alpha 
    if positive is not None: newQuery += beta * get_centroid(positive)
    if negative is not None: newQuery -= gamma * get_centroid(negative) 
    return newQuery

def iterative_loop(text, encoder, Mdb, number=1, top_positive = 3): 
    q = representation(text, encoder)
    order = get_ordered(q, Mdb)

    # top_positive = 3 
    for nth in range(number): 
        q = rocchio_relevance_feedback(q, Mdb[order[:top_positive]], Mdb[order[-top_positive:]]) #!!! set hypo 
        # q = rocchio_relevance_feedback(q, Mdb[order[:top_positive]]) #!!! set hypo 
        order = get_ordered(q, Mdb)
    # print(f"{number} [\\]") 
    return order 

iterative_loop("red people", siglip, Mdb, number=1) # [0, n-1]

# Eval Exp 

In [ ]:
from tqdm.auto import tqdm
import numpy as np

def eval_epoch(ds, selected_idx, encoder, Mdb, number_hp, breaking_rank = 1, top_positive=3):
    ranks = []
    print(f"[+] Len test: {len(selected_idx)}")
    for idx in tqdm(selected_idx):
        captions = ds[idx]["caption"].split("@")
        # Access captions for the current index
        for cap in captions:
            try:
                for N in range(0, number_hp + 1): 
                    order = iterative_loop(cap, encoder, Mdb, number=N, top_positive = top_positive)  # [0, n-1]
                    rank = np.where(order == idx)[0][0] # changed from order.index(idx) to np.where(order == idx)[0][0]
                    if(rank < breaking_rank): break 
                    
            except Exception as e:
                print(e) 
                rank = 100000 # not found

            ranks.append(rank)

    ranks = np.array(ranks)
    r1 = 100.0 * len(np.where(ranks < 1)[0]) / len(ranks)
    r5 = 100.0 * len(np.where(ranks < 5)[0]) / len(ranks)
    r10 = 100.0 * len(np.where(ranks < 10)[0]) / len(ranks)
    medr = np.floor(np.median(ranks)) + 1
    meanr = ranks.mean() + 1
    
    print(f"[+] Text to image ({len(ranks)}): Number={number_hp}")
    print("\tTop_positive: {}".format(top_positive))
    print("\tBreaking_rank: {}".format(breaking_rank))
    print("\tR@1: {:.4f}".format(r1))
    print("\tR@5: {:.4f}".format(r5))
    print("\tR@10: {:.4f}".format(r10))
    print("\tMed r: {:.4f}".format(medr))
    print("\tMean r: {:.4f}".format(meanr))

    return (r1, r5, r10, medr, meanr)


selected_idx = [i for i in range(5001, len(source_data)) if i % 300 == 0]
print(len(selected_idx))
selected_idx += [i for i in range(0, 5001) if i % 50 == 0]
print(len(selected_idx))
eval_epoch(source_data, [0, 1], siglip, Mdb, number_hp=1)

In [ ]:
from tqdm.auto import tqdm
import numpy as np
import signal
import time

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Function call timed out")

def eval_epoch_with_timeout(ds, selected_idx, encoder, Mdb, number_hp, timeout_seconds=60 * 60, breaking_rank = 1, top_positive=3):
    # Set the signal handler and a 2-minute alarm
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_seconds)
    
    try:
        return eval_epoch(ds, selected_idx, encoder, Mdb, number_hp, breaking_rank = breaking_rank, top_positive= top_positive) #!!! original eval_epoch function
    except TimeoutException:
        print("Evaluation timed out after 2 minutes")
        return None # or some default value
    finally:
        signal.alarm(0) # Disable the alarm


top_positive_list = [3, 5, 10, 15]
breaking_rank_list = [1, 5, 10, 15]

for top_positive in top_positive_list:
    for breaking_rank in breaking_rank_list:

        lis = [] 
        for number in range(0, 3):
            tup = eval_epoch_with_timeout(source_data, selected_idx, siglip, Mdb, number_hp=number, breaking_rank = breaking_rank, top_positive=top_positive)
            lis.append(tup)
